# Usando a ResNet50 para treinar com o DS

https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet50/ResNet50 </br>
https://chroniclesofai.com/transfer-learning-with-keras-resnet-50/

In [ ]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Rescaling
from tensorflow.keras import optimizers
from tensorflow.keras.utils import image_dataset_from_directory

!pip install pydicom
import numpy as np
import pydicom
from PIL import Image
import os

import cv2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


# Ajustando o DS

## Baixando o DS

In [ ]:
!kaggle datasets download -d pedroamaro/smaller-rsna-ds-train-test

100% 7.25G/7.25G [04:02<00:00, 51.6MB/s]
100% 7.25G/7.25G [04:02<00:00, 32.1MB/s]


In [ ]:
!unzip /content/smaller-rsna-ds-train-test.zip | awk 'BEGIN {ORS=" "} {if(NR%1000==0)print "."}'
!rm /content/smaller-rsna-ds-train-test.zip

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [ ]:
print('Numero de imagens: NO')
!ls /content/small_rsna_ds_train/no | wc -l
print('Numero de imagens: YES')
!ls /content/small_rsna_ds_train/yes | wc -l

Numero de imagens: NO
20047
Numero de imagens: YES
3287


## Salvando como JPG e aplicando full windowing

In [ ]:
!mkdir small_rsna_ds_as_jpg
!mkdir /content/small_rsna_ds_as_jpg/no
!mkdir /content/small_rsna_ds_as_jpg/yes

!mkdir test_as_jpg
!mkdir test_as_jpg/no
!mkdir test_as_jpg/yes

mkdir: cannot create directory ‘small_rsna_ds_as_jpg’: File exists
mkdir: cannot create directory ‘/content/small_rsna_ds_as_jpg/no’: File exists
mkdir: cannot create directory ‘/content/small_rsna_ds_as_jpg/yes’: File exists
mkdir: cannot create directory ‘test_as_jpg’: File exists
mkdir: cannot create directory ‘test_as_jpg/no’: File exists
mkdir: cannot create directory ‘test_as_jpg/yes’: File exists


In [ ]:
def segment_circle(windowed):
    original = windowed.copy().astype("uint8")
    mask = np.zeros(original.shape, dtype=np.uint8)
    gray = original
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    close = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=5)
    cnts = cv2.findContours(close, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        area = cv2.contourArea(c)
        if len(approx) > 4 and area > 10000 and area < 500000:
            ((x, y), r) = cv2.minEnclosingCircle(c)
            cv2.circle(mask, (int(x), int(y)), int(r), (255, 255, 255), -1)
            cv2.circle(original, (int(x), int(y)), int(r), (36, 255, 12), 0)
    x,y,w,h = cv2.boundingRect(mask)
    mask_ROI = mask[y:y+h, x:x+w]
    image_ROI = original[y:y+h, x:x+w]
    result = cv2.bitwise_and(image_ROI, image_ROI, mask=mask_ROI)
    return result, (x, y, w, h)

def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(img, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img


def preprocess_and_segment(path, make_correct=True):    
    dcm = pydicom.dcmread(path)
    if make_correct and (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)

    sample = dcm.pixel_array.astype("float32") * dcm.RescaleSlope + dcm.RescaleIntercept

    brain_window = window_image(sample, 40, 80)
    dural_window = window_image(sample, 80, 200)
    bone_window = window_image(sample, 600, 2800)

    brain_window, (x,y,w,h) = segment_circle(brain_window)
    if brain_window is None:
        return np.zeros((512, 512, 3))
    dural_window = dural_window[y:y+h, x:x+w]
    bone_window = bone_window[y:y+h, x:x+w]
    
    brain_window = (brain_window - (0.)) / 80.
    dural_window = (dural_window - (-20.)) / 200.
    bone_window = (bone_window - (-1200.)) / 2800.
    img_3ch = np.dstack([brain_window, dural_window, bone_window]).astype("float32")

    return img_3ch


def save_W_as_jpg(path, id, dest_folder):

  im_3ch = preprocess_and_segment(path)

  im = window_image(im_3ch, 40, 80)

  rescaled_im = (np.maximum(im, 0)/im.max())*255.0
  final_im = np.uint8(rescaled_im)

  final_im = Image.fromarray(final_im)
  final_im.save(dest_folder+id+'.jpg')

### Para o conjunto Train

In [ ]:
train_path = "/content/small_rsna_ds_train/yes/"
folder = os.listdir("/content/small_rsna_ds_train/yes/")
dest_folder = "/content/small_rsna_ds_as_jpg/yes/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
train_path = "/content/small_rsna_ds_train/no/"
folder = os.listdir("/content/small_rsna_ds_train/no/")
dest_folder = "/content/small_rsna_ds_as_jpg/no/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
print('Numero de imagens: NO')
!ls /content/small_rsna_ds_as_jpg/no/ | wc -l
print('Numero de imagens: YES')
!ls /content/small_rsna_ds_as_jpg/yes/ | wc -l

Numero de imagens: NO
20047
Numero de imagens: YES
3287


### Para o conjunto Test

In [ ]:
train_path = "/content/small_rsna_ds_test/yes/"
folder = os.listdir("/content/small_rsna_ds_test/yes/")
dest_folder = "/content/test_as_jpg/yes/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
train_path = "/content/small_rsna_ds_test/no/"
folder = os.listdir("/content/small_rsna_ds_test/no/")
dest_folder = "/content/test_as_jpg/no/"

for id_ in folder:
  img_id = id_[:12]
  save_W_as_jpg(train_path + f"{id_}", img_id, dest_folder)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
print('Numero de imagens: NO')
!ls /content/test_as_jpg/no/ | wc -l
print('Numero de imagens: YES')
!ls /content/test_as_jpg/yes/ | wc -l

Numero de imagens: NO
10022
Numero de imagens: YES
1643


# Carregando o dataset

In [ ]:
train_data = image_dataset_from_directory(
    "/content/small_rsna_ds_as_jpg",
     labels='inferred',
     validation_split=1/3,
     subset="training",
     seed=123,
     batch_size = 64,
     color_mode = "rgb"
     )

Found 23334 files belonging to 2 classes.
Using 15556 files for training.


In [ ]:
val_data = image_dataset_from_directory(
    "/content/small_rsna_ds_as_jpg",
     labels='inferred',
     validation_split=1/3,
     subset="validation",
     seed=123,
     batch_size = 64,
     color_mode = "rgb"
)

Found 23334 files belonging to 2 classes.
Using 7778 files for validation.


In [ ]:
test_data = image_dataset_from_directory(
    "/content/test_as_jpg",
     labels='inferred',
     label_mode="int",
     batch_size = 64,
     color_mode = "rgb"
     )

Found 11665 files belonging to 2 classes.


In [ ]:
class_names = train_data.class_names

## Normalizando e Pegando o tamanho de entrada

In [ ]:
normalization_layer = Rescaling(1./255)

normalized_data = train_data.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_data))

normalized_val_data = val_data.map(lambda x, y: (normalization_layer(x), y))
val_image_batch, val_labels_batch = next(iter(normalized_val_data))

normalized_test_data = test_data.map(lambda x, y: (normalization_layer(x), y))
test_image_batch, test_labels_batch = next(iter(normalized_test_data))

# ResNet

In [ ]:
keras.backend.clear_session()

In [ ]:
resnet_model = Sequential()

resnet = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    input_shape=image_batch.shape[1:],
    pooling='avg',
    weights='imagenet',
    classes=2,
)

resnet_model.add(resnet)

resnet_model.add(Flatten())

resnet_model.add(Dense(512, activation="relu"))

resnet_model.add(Dropout(0.5))

resnet_model.add(Dense(1, activation="sigmoid"))

resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                                 
Total params: 24,637,313
Trainable params: 24,584,193
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
neg = 20047
pos = 3287
total = pos+neg

w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)

print('Peso para a classe 0: {:.2f}'.format(w0))
print('Peso para a classe 1: {:.2f}'.format(w1))

Peso para a classe 0: 0.58
Peso para a classe 1: 3.55


In [ ]:
resnet_model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00001), metrics=[keras.metrics.AUC(), 'accuracy'])

In [ ]:
run_hist = resnet_model.fit(normalized_data, validation_data=normalized_val_data, epochs=5, class_weight={0:w0, 1:w1}, 
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/5
244/244 [==============================] - 243s 903ms/step - loss: 0.5952 - auc: 0.7535 - accuracy: 0.6916 - val_loss: 0.4304 - val_auc: 0.4912 - val_accuracy: 0.8581
Epoch 2/5
244/244 [==============================] - 218s 893ms/step - loss: 0.4399 - auc: 0.8744 - accuracy: 0.8098 - val_loss: 0.4275 - val_auc: 0.5240 - val_accuracy: 0.8581
Epoch 3/5
244/244 [==============================] - 219s 894ms/step - loss: 0.3373 - auc: 0.9291 - accuracy: 0.8726 - val_loss: 0.4276 - val_auc: 0.6268 - val_accuracy: 0.8590
Epoch 4/5
244/244 [==============================] - 219s 895ms/step - loss: 0.2537 - auc: 0.9609 - accuracy: 0.9177 - val_loss: 0.3334 - val_auc: 0.8311 - val_accuracy: 0.8795
Epoch 5/5
244/244 [==============================] - 219s 894ms/step - loss: 0.2060 - auc: 0.9736 - accuracy: 0.9370 - val_loss: 0.4169 - val_auc: 0.8720 - val_accuracy: 0.8623


In [ ]:
results = resnet_model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 47s 250ms/step - loss: 0.4187 - auc: 0.8767 - accuracy: 0.8651
[0.4186899662017822, 0.8766594529151917, 0.865066409111023]


In [ ]:
run_hist = resnet_model.fit(normalized_data, validation_data=normalized_val_data, epochs=10, class_weight={0:w0, 1:w1}, 
  callbacks=[keras.callbacks.EarlyStopping(
                  monitor="val_auc",
                  min_delta=0,
                  patience=5,
                  verbose=1,
                  mode="auto",
                  baseline=None,
                  restore_best_weights=True,
              )])

Epoch 1/10
244/244 [==============================] - 226s 899ms/step - loss: 0.6047 - auc: 0.7471 - accuracy: 0.6803 - val_loss: 0.5590 - val_auc: 0.5693 - val_accuracy: 0.8458
Epoch 2/10
244/244 [==============================] - 219s 896ms/step - loss: 0.4404 - auc: 0.8733 - accuracy: 0.8147 - val_loss: 0.4777 - val_auc: 0.6668 - val_accuracy: 0.8321
Epoch 3/10
244/244 [==============================] - 219s 894ms/step - loss: 0.3448 - auc: 0.9243 - accuracy: 0.8761 - val_loss: 0.5005 - val_auc: 0.7776 - val_accuracy: 0.7701
Epoch 4/10
244/244 [==============================] - 218s 892ms/step - loss: 0.2555 - auc: 0.9604 - accuracy: 0.9196 - val_loss: 0.5501 - val_auc: 0.8388 - val_accuracy: 0.7840
Epoch 5/10
244/244 [==============================] - 219s 894ms/step - loss: 0.1796 - auc: 0.9803 - accuracy: 0.9522 - val_loss: 0.6495 - val_auc: 0.8822 - val_accuracy: 0.8035
Epoch 6/10
244/244 [==============================] - 218s 892ms/step - loss: 0.1350 - auc: 0.9874 - accuracy:

In [ ]:
results = resnet_model.evaluate(normalized_test_data)
print(results)

183/183 [==============================] - 46s 249ms/step - loss: 0.6581 - auc: 0.8814 - accuracy: 0.7976
[0.6580597758293152, 0.881447434425354, 0.7975996732711792]
